<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR Data Analysis ⚡⚡</h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">

  


<h1 style="color:#D4AF37"> What's the Business Problem? 💰</h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages. 

There are four key types of outages:

● Consequential

● Forced 

● Opportunistic 

● Planned 

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed:

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>



In [2]:
%%capture
!pip install ipython-sql sqlalchemy
import sqlalchemy
sqlalchemy.create_engine("sqlite:///AEMR.db")
%load_ext sql
%sql sqlite:///AEMR.db

In [3]:
%%js
require(['notebook/js/codecell'], function (codecell) {
    codecell.CodeCell.options_default.highlight_modes['magic_text/x-mssql'] = { 'reg': [/%?%sql/] };
    Jupyter.notebook.events.one('kernel_ready.Kernel', function () {
        Jupyter.notebook.get_cells().map(function (cell) {
            if (cell.cell_type == 'code') { cell.auto_highlight(); }
        });
    });
});


<IPython.core.display.Javascript object>

<h3 style="color:#D4AF37"> ⚡ Part I. Energy Stability & Market Outages ⚡ </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>
<b>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>

<u style="color:Maroon"> Please note that throughout the entire case study, we are interested ONLY in the Outages where Status = Approved. 

<h3 style="color:#D4AF37"> Question One </h3> 

<b> Write a SQL Statement to `COUNT` the number of valid (i.e. `Status = Approved`) Outage Events sorted by their respective `Outage_Reason` (i.e. `Forced`, `Consequential`, `Scheduled`, `Opportunistic`) over the 2016 & 2017 Periods. </b>


In [4]:
%%sql
SELECT
COUNT(*) as Total_Number_Outages, Outage_Reason, Year
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Outage_Reason, Year
LIMIT 10

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016
127,Consequential,2017
1264,Forced,2016
1622,Forced,2017
106,Opportunistic Maintenance (Planned),2016
102,Opportunistic Maintenance (Planned),2017
380,Scheduled (Planned),2016
320,Scheduled (Planned),2017


<h3 style="color:#D4AF37"> Question Two </h3>



<b> i) Write a SQL Statement showing the `Total` of all Outage Types (Forced, Consequential, Scheduled, Opportunistic) where the `Status = Approved`, that occurred for both 2016 and 2017, grouped by `Year` and `Month`.  per month (i.e. 1 – 12). Order by `Year`, `Month`, `Total_Number_Outages` in Descending Order.


In [42]:
%%sql
SELECT
COUNT(*) as Total_Number_Outages, Year, Month
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Year, Month
LIMIT 10

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Year,Month
191,2016,1
227,2016,2
136,2016,3
134,2016,4
174,2016,5
168,2016,6
147,2016,7
194,2016,8
124,2016,9
124,2016,10


We've now identified a specific outage type is rapidly increasing from 2016 to 2017. However, frequency is just one aspect we care about. We also care about the <b> `duration` </b> of our ouatges.

In other words, if an outage is very short, we aren't as concerned. However, if the outage is very long, this then has the risk of threatening our energy supplies. We want to identify the problematic energy providers here.  

<h3 style="color:#D4AF37"> Question Three </h3>


Write a SQL statement that calculates 1) The `Total_Number_Outage_Events` and 2) The `Average Duration` in DAYS for each `Participant Code` and `Outage Type` over the 2016 and 2017 Period where the `Status = Approved`. 
Order by `Total_Number_Outage_Events` in Descending Order, `Reason` and `Year`.
    


In [44]:
%%sql
SELECT Participant_Code, Outage_Reason, Year,
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status= 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
ORDER BY Total_Number_Outage_Events DESC
LIMIT 10

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days
AURICON,Forced,2017,490,0.07
GW,Forced,2016,317,0.38
GW,Forced,2017,227,1.06
AURICON,Forced,2016,208,0.07
AUXC,Forced,2016,206,0.08
MELK,Forced,2017,177,2.28
TRMOS,Forced,2017,172,0.42
MELK,Forced,2016,157,0.83
PUG,Forced,2017,135,0.25
AUXC,Forced,2017,120,0.02


We've identified participants who are having many outages, as well as participants who have been offline for the longest durations.

We classify a participant based off the following criteria:
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li> 
</b>

<h3 style="color:#D4AF37"> Question Four </h3>

Classify each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification`</u> that is based off their Average Outage Duration Time. Please note that this is for all valid (i.e. `Where status = approved`) outage types (Forced, Consequential, Scheduled, Opportunistic) for <u>all</u> participant codes from 2016 to 2017. 


In [49]:
%%sql
WITH CTE1 AS (
SELECT Participant_Code, Outage_Reason, Year,
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status= 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
ORDER BY Total_Number_Outage_Events DESC
    
)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days > 1 THEN 'High Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0.5 AND 1 THEN 'Medium Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0 AND 0.5 THEN 'Low Risk'
    ELSE 'N/A'
END as Risk_Classification    
FROM CTE1
LIMIT 5;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,Low Risk
GW,Forced,2016,317,0.38,Low Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,Low Risk
AUXC,Forced,2016,206,0.08,Low Risk


<h3 style="color:#D4AF37"> Question Five </h3>

Using the above criteria for context, write a SQL Statement that <u> classifies each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification`</u> using the new classification criteria. Order the results using `Average Duration Time In Days` in descending order. 


In [51]:
%%sql
WITH CTE1 AS (
SELECT Participant_Code, Outage_Reason, Year,
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Average_Outage_Duration_In_Days
FROM AEMR_Outage_Table
WHERE Status= 'Approved'
GROUP BY Participant_Code, Outage_Reason, Year
ORDER BY Total_Number_Outage_Events DESC
    
)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days > 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events > 20 AND Outage_Reason = 'Forced' THEN 'High Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0.5 AND 1 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events BETWEEN 10 AND 20 AND Outage_Reason = 'Forced' THEN 'Medium Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0 AND 0.5 AND Outage_Reason = 'Forced' OR Total_Number_Outage_Events < 10 AND Outage_Reason = 'Forced' THEN 'Low Risk'
    ELSE 'N/A'
END as Risk_Classification    
FROM CTE1
LIMIT 20;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,Forced,2017,490,0.07,High Risk
GW,Forced,2016,317,0.38,High Risk
GW,Forced,2017,227,1.06,High Risk
AURICON,Forced,2016,208,0.07,High Risk
AUXC,Forced,2016,206,0.08,High Risk
MELK,Forced,2017,177,2.28,High Risk
TRMOS,Forced,2017,172,0.42,High Risk
MELK,Forced,2016,157,0.83,High Risk
PUG,Forced,2017,135,0.25,High Risk
AUXC,Forced,2017,120,0.02,High Risk


<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

When an energy provider provides energy to the market, they are making a commitment to the market and saying; “We will supply X amount of energy to the market under a contractual obligation.” However, in a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. <b style="color:Maroon">If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate. </b>

To ensure this doesn’t happen, the AEMR is interested in exploring the following questions:

<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?
    
<p>

    
<img src = "https://media.istockphoto.com/id/1281821795/photo/market-stock-graph-and-information-with-city-light-and-electricity-and-energy-facility-banner.jpg?s=612x612&w=0&k=20&c=RSN5LqeMW28HW10aA190_DWR5YJ5tG2wixHFPBV3uZE=">

<h3 style="color:#D4AF37"> Question Six </h3>

Calculate the proportion of Forced Outages that have occurred over the 2016 - 2017 Period.


In [59]:
%%sql
SELECT COUNT(*) as Total_Number_Outages,
SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) as Total_Number_Forced_Outage_Events, 
Year,
ROUND(SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END)*1.0/COUNT(*)*100,2) as Pct_Outage_Forced 

FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Year

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Number_Forced_Outage_Events,Year,Pct_Outage_Forced
1931,1264,2016,65.46
2171,1622,2017,74.71


It's clear to see now that `Forced Outages` are problematic for us. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.

Let's break our analysis down into Macro and Micro Analysis.
The total gives us the Overall Duration a participant is offline / has lost energy, however, it doesn't tell us how *frequently* this occurs.


<h3 style="color:#D4AF37"> Question Seven </h3>

Calculate the `Total Number of Outages`, `Total Duration In Days`and `Total Energy Lost` of all valid `Outages` for each `participant code` and `facility_code`, sorted by `Total Energy Lost` in descending order and Ordered by the YEAR Category. 
    

In [14]:
%%sql
SELECT
COUNT(*) as Total_Number_Outages,
ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) as Total_Duration_In_Days,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved' 
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Duration_In_Days DESC, Year DESC
LIMIT 5;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
70,482.58,7499.28,Scheduled (Planned),MELK,MELK_G7,2017
177,404.15,10285.4,Forced,MELK,MELK_G7,2017
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016
227,240.69,19326.56,Forced,GW,BW1_GREENWATERS_G2,2017
45,199.4,6450.0,Scheduled (Planned),GW,BW1_GREENWATERS_G2,2016


<h3 style="color:#D4AF37"> Question Eight </h3>

Calculate the `Average Duration In Days`and `Average Energy Lost` of all valid `FORCED OUTAGES` for each `participant code` and `facility_code` sorted by `Average Energy Lost` in descending order and Ordered by the YEAR Category. 


In [66]:
%%sql
SELECT
ROUND(AVG(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) as Avg_Duration_In_Days,
ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
ORDER BY Avg_Energy_Lost ASC, Year DESC
LIMIT 5;

 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
0.48,1.44,Forced,DNHR,DNHR_DENMARK_WF1,2017
0.4,1.44,Forced,DNHR,DNHR_DENMARK_WF1,2016
5.9,5.89,Forced,EUCT,GRASMERE_WF1,2016
0.08,13.27,Forced,AUXC,AUXC_WGP,2016
0.02,14.74,Forced,AUXC,AUXC_WGP,2017


<h3 style="color:#D4AF37"> Question Nine </h3>

Calculate the `Average Energy Lost` and `Total Energy Lost` for each `Facility Code` and `Participant Code` across both the 2016 and 2017 periods when the `Outage_Reason` is set to Forced. Upon completion of this we calculate the <u> percentage </u> of energy lost due to forced outages for each `Facility_Code`.


In [68]:
%%sql
SELECT
ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) as Pct_Energy_Loss, 
Outage_Reason, 
Participant_Code,
Facility_Code,
Year

FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Outage_Reason, Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC, Year DESC
LIMIT 5

 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016


<h3 style="color:#D4AF37"> Question Ten </h3>

<b>Having identified the top 3 participants by Total Energy Loss being `GW`, `MELK` and `Auricon`; We write a SQL Statement calculating the `Total_Energy_Lost` each of these three `Participant_Codes` and the `Facility_Code`. Additionally, identifying the `Description_Of_Outage` associated with the highest `Total_Energy_Lost` for each of the `Participant_Codes` and `Facility_Code` for each of the three participants. 
    
<u>Lastly, we calculate the percentage of Energy Loss, attributed to these reasons!</u> </b>

In [71]:
%%sql
SELECT *
FROM(
SELECT 
Participant_Code,
Facility_Code,
Description_Of_Outage,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW)/(SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) as Pct_Forced,
RANK() OVER(PARTITION BY Participant_Code, Facility_Code ORDER BY SUM(Energy_Lost_MW) DESC) as Ranking
FROM AEMR_Outage_Table
WHERE Participant_Code IN ('GW', 'MELK', 'AURICON') AND Status = 'Approved' and Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Description_Of_Outage
ORDER BY Participant_Code, Total_Energy_Lost DESC)

WHERE Ranking = 1 


 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,Total_Energy_Lost,Pct_Forced,Ranking
AURICON,AURICON_PNJ_U1,Full unit trip,6033.87,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,28687.54,18.84,1
MELK,MELK_G7,Safety Issues,1100.0,0.72,1


Which Market Particpants are responsible for the most outages?

In [15]:
%%sql
SELECT Participant_Code,
COUNT(*) as Total_Number_Outages
FROM AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY Participant_Code
ORDER BY Total_Number_Outages DESC


 * sqlite:///AEMR.db
Done.


Participant_Code,Total_Number_Outages
AURICON,875
GW,672
MELK,536
AUXC,331
PJRH,288
TRMOS,247
PUG,196
KORL,195
PMC,168
ENRG,123


In [58]:
%%sql
SELECT 
Participant_Code, 
COUNT(*) AS Total_Number_Outages,
ROUND(CAST(COUNT(*) AS FLOAT) / (SELECT COUNT(*) FROM AEMR_Outage_Table WHERE Year BETWEEN 2016 AND 2017) * 100, 2) AS Pct_Outage
FROM 
AEMR_Outage_Table
WHERE 
Status = 'Approved' AND Year BETWEEN 2016 AND 2017
GROUP BY 
Participant_Code
ORDER BY Pct_Outage DESC

 * sqlite:///AEMR.db
Done.


Participant_Code,Total_Number_Outages,Pct_Outage
AURICON,875,18.8
GW,672,14.44
MELK,536,11.51
AUXC,331,7.11
PJRH,288,6.19
TRMOS,247,5.31
PUG,196,4.21
KORL,195,4.19
PMC,168,3.61
ENRG,123,2.64


Which Market Participants have lost the most energy?

In [29]:
%%sql
SELECT 
Participant_Code, 
ROUND(SUM(Energy_Lost_MW),2) AS total_energy_lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(Energy_Lost_MW) FROM AEMR_Outage_Table) * 100, 2) AS percentage_of_energy_lost
FROM 
AEMR_Outage_Table
WHERE 
Status = 'Approved' AND Year BETWEEN 2016 AND 2017
GROUP BY 
Participant_Code
ORDER BY 
total_energy_lost DESC


 * sqlite:///AEMR.db
Done.


Participant_Code,total_energy_lost,percentage_of_energy_lost
AURICON,52243.66,17.11
GW,50636.89,16.59
MELK,46966.09,15.38
PMC,24205.39,7.93
PJRH,20555.71,6.73
KORL,14253.0,4.67
COLLGAR,13944.52,4.57
PUG,8349.57,2.73
TRMOS,6585.9,2.16
AUXC,4629.9,1.52


Which Market Participants have been offline for the longest durations?

In [15]:
%%sql
SELECT 
Participant_Code,
COUNT(*) AS Total_Number_Outages,
ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))),2) as Total_Duration_In_Days
FROM 
AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
Participant_Code
ORDER BY 
Total_Duration_In_Days DESC


 * sqlite:///AEMR.db
Done.


Participant_Code,Total_Number_Outages,Total_Duration_In_Days
MELK,536,1441.37
GW,672,644.02
ENRG,123,386.37
PJRH,288,353.54
KORL,195,324.02
AURICON,875,216.29
COLLGAR,117,213.04
PMC,168,138.67
PUG,196,106.04
TRMOS,247,100.9


What is the most common outage type?

In [37]:
%%sql
SELECT 
Outage_Reason, 
COUNT(*) AS outage_count
FROM 
AEMR_Outage_Table
WHERE Status = 'Approved'
GROUP BY 
Outage_Reason
ORDER BY 
outage_count DESC



 * sqlite:///AEMR.db
Done.


Outage_Reason,outage_count
Forced,2886
Scheduled (Planned),700
Consequential,308
Opportunistic Maintenance (Planned),208


Forced Outage 2016 vs 2017

In [39]:
%%sql
SELECT 
Month,
SUM(CASE WHEN Year = 2016 THEN 1 ELSE 0 END) AS '2016_Outages',
SUM(CASE WHEN Year = 2017 THEN 1 ELSE 0 END) AS '2017_Outages'
FROM 
AEMR_Outage_Table
WHERE 
Outage_Reason = (SELECT Outage_Reason FROM AEMR_Outage_Table GROUP BY Outage_Reason ORDER BY COUNT(*) DESC LIMIT 1) AND Status = 'Approved'
GROUP BY 
Month


 * sqlite:///AEMR.db
Done.


Month,2016_Outages,2017_Outages
1,134,70
2,149,97
3,94,53
4,86,73
5,101,105
6,112,172
7,80,182
8,152,170
9,96,137
10,67,207


How many outages are approved in comparison to those that are canceled?

In [41]:
%%sql
SELECT 
Status, 
COUNT(*) AS outage_count,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER(), 2) AS percentage
FROM 
AEMR_Outage_Table
GROUP BY 
Status



 * sqlite:///AEMR.db
Done.


Status,outage_count,percentage
Accepted,9,0.19
Approved,4102,88.12
Cancelled By Market Participant,522,11.21
Cancelled By System Management,6,0.13
Not Accepted,7,0.15
Rejected,9,0.19


When contrasting Energy Losses, Duration and Outage Frequency – do we
have separate market participants which contribute the most for each of
these areas?

In [18]:
%%sql
SELECT 
Participant_Code,
ROUND(SUM(Energy_Lost_MW),2) AS total_energy_losses,
ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))), 2) AS total_duration_in_days,
COUNT(*) AS total_outages
FROM 
AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced'  
GROUP BY 
Participant_Code
ORDER BY 
total_energy_losses DESC,
total_duration_in_days DESC,
total_outages DESC


 * sqlite:///AEMR.db
Done.


Participant_Code,total_energy_losses,total_duration_in_days,total_outages
GW,35077.94,361.29,544
AURICON,32335.82,48.71,698
MELK,24056.47,533.75,334
PMC,14741.52,35.62,109
PJRH,10720.8,158.92,153
KORL,8720.0,111.79,129
COLLGAR,7107.91,94.44,74
TRMOS,6249.1,84.52,237
PUG,4927.57,55.56,159
AUXC,4502.9,18.15,326


Forced Outage By Year 2016

In [10]:
%%sql
SELECT 
  
  outage_reason,
  strftime('%Y-%m', Start_Time) AS year_month,
  ROUND(SUM(Energy_Lost_MW),2) AS total_energy_losses,
  ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))), 2) AS total_duration_in_days,
  COUNT(*) AS total_outages
FROM 
  AEMR_Outage_Table
WHERE 
  Status = 'Approved' AND Year = 2016 AND outage_reason = 'Forced'   
GROUP BY 
  year_month
ORDER BY 
  year_month


 * sqlite:///AEMR.db
Done.


Outage_Reason,year_month,total_energy_losses,total_duration_in_days,total_outages
Forced,2016-01,9361.65,26.85,134
Forced,2016-02,8038.62,19.71,149
Forced,2016-03,5617.4,64.96,94
Forced,2016-04,5211.28,43.75,86
Forced,2016-05,4404.3,49.15,101
Forced,2016-06,5521.5,59.23,112
Forced,2016-07,4116.08,37.77,80
Forced,2016-08,6434.68,66.63,152
Forced,2016-09,6959.25,63.69,96
Forced,2016-10,4018.39,69.44,67


Forced Outage By Year 2017

In [56]:
%%sql
SELECT 
  
  outage_reason,
  strftime('%Y-%m', Start_Time) AS year_month,
  ROUND(SUM(Energy_Lost_MW),2) AS total_energy_losses,
  ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))), 2) AS total_duration_in_days,
  COUNT(*) AS total_outages
FROM 
  AEMR_Outage_Table
WHERE 
  Status = 'Approved' AND Year = 2017 AND outage_reason = 'Forced'   
GROUP BY 
  year_month
ORDER BY 
  year_month
 

 * sqlite:///AEMR.db
Done.


Outage_Reason,year_month,total_energy_losses,total_duration_in_days,total_outages
Forced,2017-01,3826.34,34.85,70
Forced,2017-02,6437.5,60.21,97
Forced,2017-03,4314.29,91.21,53
Forced,2017-04,5574.1,49.29,73
Forced,2017-05,4007.7,31.44,105
Forced,2017-06,8014.19,82.92,172
Forced,2017-07,9513.66,99.46,182
Forced,2017-08,10233.84,83.31,170
Forced,2017-09,5813.03,262.38,137
Forced,2017-10,5549.84,51.35,207


In [20]:
%%sql
SELECT
Participant_Code,
strftime('%Y-%m', Start_Time) AS year_month,
  ROUND(SUM(Energy_Lost_MW),2) AS total_energy_losses,
  ROUND(SUM(ABS((JULIANDAY(End_Time)-JULIANDAY(Start_Time)))), 2) AS total_duration_in_days,
COUNT(*) as Total_Number_Outages, Outage_Reason
FROM AEMR_Outage_Table
WHERE Status = 'Approved' AND Outage_Reason = 'Forced' AND Participant_Code IN ('AURICON', 'GW', 'MELK')
GROUP BY year_month, Participant_Code 
ORDER BY year_month, Participant_Code


 * sqlite:///AEMR.db
Done.


Participant_Code,year_month,total_energy_losses,total_duration_in_days,Total_Number_Outages,Outage_Reason
AURICON,2016-01,1215.55,0.29,27,Forced
GW,2016-01,319.61,1.02,9,Forced
MELK,2016-01,1696.45,1.33,16,Forced
AURICON,2016-02,435.43,4.48,27,Forced
GW,2016-02,702.49,0.92,11,Forced
MELK,2016-02,1557.04,5.25,26,Forced
AURICON,2016-03,73.95,0.0,6,Forced
GW,2016-03,2758.92,1.15,49,Forced
MELK,2016-03,880.06,30.85,10,Forced
AURICON,2016-04,897.98,0.0,5,Forced



<img src = https://www.energymagazine.com.au/wp-content/uploads/2021/05/shutterstock_1888482466-e1620278098312.jpg>
